การติดตั้ง

In [2]:
# pip install -U torch sahi yolov5
import os
os.getcwd()

'c:\\bankprp\\Projects\\Palm_detection'

In [3]:
from sahi.utils.yolov5 import download_yolov5s6_model
from sahi.model import Yolov5DetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image

c:\Users\peera\anaconda3\envs\SAHI\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
yolov5_model_path = r'C:\bankprp\Projects\Palm_detection\best_weight.pt'
download_yolov5s6_model(destination_path=yolov5_model_path)

In [5]:
# ตรวจสอบ gpu
import torch
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

detection_model = Yolov5DetectionModel(
    model_path = yolov5_model_path,
    confidence_threshold = 0.3,
    device = device
)

Sliced Inference with a YOLOv5 Model

In [7]:
result = get_sliced_prediction(
    r"C:\bankprp\Projects\Palm_detection\Palm_2.tif",
    detection_model,
    slice_height = 1024,
    slice_width = 1024,
    overlap_height_ratio = 0.2,
    overlap_width_ratio = 0.2
)

Performing prediction on 990 number of slices.


In [8]:
result.to_coco_annotations()

[{'image_id': None,
  'bbox': [12720, 5834, 255, 252],
  'score': 0.8849069476127625,
  'category_id': 15,
  'category_name': 'palm',
  'segmentation': [],
  'iscrowd': 0,
  'area': 64260},
 {'image_id': None,
  'bbox': [22247, 16361, 314, 299],
  'score': 0.8810576796531677,
  'category_id': 15,
  'category_name': 'palm',
  'segmentation': [],
  'iscrowd': 0,
  'area': 93886},
 {'image_id': None,
  'bbox': [14901, 24219, 248, 258],
  'score': 0.8810309171676636,
  'category_id': 15,
  'category_name': 'palm',
  'segmentation': [],
  'iscrowd': 0,
  'area': 63984},
 {'image_id': None,
  'bbox': [15849, 23849, 245, 237],
  'score': 0.865771472454071,
  'category_id': 15,
  'category_name': 'palm',
  'segmentation': [],
  'iscrowd': 0,
  'area': 58065},
 {'image_id': None,
  'bbox': [6994, 15971, 227, 215],
  'score': 0.860384464263916,
  'category_id': 15,
  'category_name': 'palm',
  'segmentation': [],
  'iscrowd': 0,
  'area': 48805},
 {'image_id': None,
  'bbox': [13155, 23707, 256,

In [9]:
import numpy as np
row_column = []
for c in result.to_coco_annotations() :
    xmin, ymin, width, height = c['bbox']
    center_x = xmin + width/2
    center_y = ymin + height/2
    row_column.append((center_x, center_y))


Rasterio

In [10]:
# Install GDAL and rasterio wheel from https://www.lfd.uci.edu/~gohlke/pythonlibs/ then pip3 install <filename>
import rasterio
import rasterio.features
import rasterio.warp
import csv

In [11]:
with rasterio.open(r'C:\bankprp\Projects\Palm_detection\Palm_2.tif') as dataset :
    print(dataset.profile)
    aff = dataset.transform

06/30/2022 16:26:24 - WARNING - rasterio._env -   CPLE_AppDefined in PROJ: proj_create_from_database: C:\Users\peera\anaconda3\envs\SAHI\Library\share\proj\proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
CPLE_AppDefined in PROJ: proj_create_from_database: C:\Users\peera\anaconda3\envs\SAHI\Library\share\proj\proj.db lacks DATABASE.LAYOUT.VERSION.MAJOR / DATABASE.LAYOUT.VERSION.MINOR metadata. It comes from another PROJ installation.
06/30/2022 16:26:24 - WARNING - rasterio._env -   CPLE_AppDefined in The definition of projected CRS EPSG:32647 got from GeoTIFF keys is not the same as the one from the EPSG registry, which may cause issues during reprojection operations. Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys), or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.
CPLE_AppDefined in The definition of projected CR

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 24642, 'height': 27037, 'count': 4, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 47N",GEOGCS["WGS 84",DATUM["World Geodetic System 1984",SPHEROID["WGS 84",6378137,298.257223563]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",99],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'), 'transform': Affine(0.03194999999999884, 0.0, 712957.49175,
       0.0, -0.03195000000000545, 1571104.3053500003), 'tiled': False, 'interleave': 'pixel'}


In [12]:
with open(r'C:\bankprp\Projects\Palm_detection\palm_coor.csv', 'w') as f :
    writer = csv.writer(f)
    writer.writerow(['Latitude', 'Longtitude'])
    for c in row_column:
        writer.writerow(aff*c)